In [1]:
import os
import sys
import zipfile
import pandas as pd
import numpy as np
import plotly.express as px
import matplotlib.pyplot as plt

%matplotlib inline

sys.path.append('../')
from src.data_preprocess import DataPreprocessor

pd.set_option('display.max_colwidth', 2000)

%load_ext autoreload
%autoreload 2

In [2]:
train_df = pd.read_csv('../input/train.csv')
train_df.head(2)

,listing_id,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,...,furnishing,available_unit_types,total_num_units,property_details_url,lat,lng,elevation,subzone,planning_area,price
0,122881,hdb flat for sale in 866 yishun street 81,sembawang / yishun (d27),866 yishun street 81,hdb 4 rooms,NaN,1988.0,3.0,2.0,1115,...,unspecified,NaN,116.0,https://www.99.co/singapore/hdb/866-yishun-street-81-adxawp85buupmsq7gwdjverc,1.414399,103.837196,0,yishun south,yishun,514500.0
1,259374,hdb flat for sale in 506b serangoon north avenue 4,hougang / punggol / sengkang (d19),hdb-serangoon estate,hdb,99-year leasehold,1992.0,4.0,2.0,1575,...,unspecified,"1, 2, 3, 4, 5, 6 br",NaN,https://www.99.co/singapore/hdb/hdbserangoon-estate-demrpm6ryc3l9buf846erprb,1.372597,103.875625,0,serangoon north,serangoon,995400.0


In [3]:
train_df_clean = DataPreprocessor.data_preprocessing_v1(train_df, test=False, uncertain=False, drop_na=False, remove_original_attributes=False)

Processed function: Function 'preprocess_available_unit_types' executed in 11.2567s & Processing function: preprocess_planning_area: 100%|██████████| 14/14 [01:52<00:00,  8.01s/it]


## sg-commerical-centres

In [7]:
sg_commercial_centres_df = pd.read_csv('../input/auxiliary-data/sg-commerical-centres.csv')
sg_commercial_centres_df.loc[sg_commercial_centres_df['type']=='IEPB', 'type'] = 'IEBP'
sg_commercial_centres_df.head(2)

,name,type,lat,lng,subzone,planning_area
0,Central Business District,CR,1.286768,103.854529,clifford pier,downtown core
1,Jurong Lake District,CR,1.334085,103.734513,lakeside (business),jurong east


In [8]:
test_df = DataPreprocessor.preprocess_commercial_center(
    train_df_clean,
    sg_commerical_centres_df,
    test=False
)

CR
IEBP
BN
IHL


In [18]:
test_df.head(2)

,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,...,furnishing_cat_3,number_of_types_available,has_studio,min_br_available,max_br_available,planning_area_cat_0,pri_sch_dist,pri_sch_name,number_of_close_pri_sch,pri_sch_cat_0
0,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,0.0,4,1,1,3,0.0,0.572565,St. Joseph's Institution Junior,0,154.0
1,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,1.0,4,1,1,3,0.0,0.572565,St. Joseph's Institution Junior,0,154.0


In [ ]:
train_df_clean = train_df_clean.merge(test_df, left_on=['lat','lng'], right_on=['lat','lng'])
train_df_clean.head()

# sg-primary-schools.csv

In [10]:
sg_primary_schools_df = pd.read_csv('../input/auxiliary-data/sg-primary-schools.csv')
sg_primary_schools_df.head(2)

,name,lat,lng,subzone,planning_area
0,Admiralty Primary School,1.442941,103.800345,woodlands east,serangoon
1,Ahmad Ibrahim Primary School,1.433849,103.832710,yishun west,serangoon


In [13]:
test_df = DataPreprocessor.preprocess_primary_school(
    train_df_clean,
    sg_primary_schools_df,
    test=False
)

In [19]:
test_df.head(2)

,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,...,furnishing_cat_3,number_of_types_available,has_studio,min_br_available,max_br_available,planning_area_cat_0,pri_sch_dist,pri_sch_name,number_of_close_pri_sch,pri_sch_cat_0
0,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,0.0,4,1,1,3,0.0,0.572565,St. Joseph's Institution Junior,0,154.0
1,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,1.0,4,1,1,3,0.0,0.572565,St. Joseph's Institution Junior,0,154.0


# sg-secondary-schools.csv

In [17]:
sg_secondary_schools_df = pd.read_csv('../input/auxiliary-data/sg-secondary-schools.csv')
sg_secondary_schools_df.head(2)

,name,lat,lng,subzone,planning_area
0,Admiralty Secondary School,1.445912,103.802908,woodlands east,woodlands
1,Ahmad Ibrahim Secondary School,1.436095,103.830055,yishun west,yishun


In [22]:
test_df = DataPreprocessor.preprocess_secondary_school(
    train_df_clean,
    sg_secondary_schools_df,
    test=False
)

In [23]:
test_df.head(2)

,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,...,furnishing_cat_3,number_of_types_available,has_studio,min_br_available,max_br_available,planning_area_cat_0,sec_sch_dist,sec_sch_name,number_of_close_sec_sch,sec_sch_cat_0
0,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,0.0,4,1,1,3,0.0,0.634515,Anglo-Chinese School (Barker Road),0,127.0
1,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,1.0,4,1,1,3,0.0,0.634515,Anglo-Chinese School (Barker Road),0,127.0


# sg-shopping-malls.csv

In [24]:
sg_shopping_mall_df = pd.read_csv('../input/auxiliary-data/sg-shopping-malls.csv')
sg_shopping_mall_df.head(2)

,name,lat,lng,subzone,planning_area
0,10 AM,1.275568,103.863591,marina south,marina south
1,313@Somerset,1.301013,103.838540,somerset,orchard


In [25]:
test_df = DataPreprocessor.preprocess_shopping_mall(
    train_df_clean,
    sg_shopping_mall_df,
    test=False
)

In [26]:
test_df.head(2)

,title,address,property_name,property_type,tenure,built_year,num_beds,num_baths,size_sqft,floor_level,...,furnishing_cat_3,number_of_types_available,has_studio,min_br_available,max_br_available,planning_area_cat_0,shopping_mall_dist,shopping_mall_name,number_of_close_shopping_mall,shopping_mall_cat_0
0,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,0.0,4,1,1,3,0.0,0.353193,United Square,1,112.0
1,1 bed condo for sale in 10 evelyn,10 evelyn road,10 evelyn,Condo,freehold,2022.0,1.0,1.0,495,NaN,...,1.0,4,1,1,3,0.0,0.353193,United Square,1,112.0
